In [ ]:
# go through every sleep event file
# add the start and end time and int value

#sort all of the data points
#fill in the -1 values

#save

In [2]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeHypnoDfParquet

import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

exportPath = exportsDataPath + "/whoop/WHOOP Production-export/Health/"

files = [f for f in os.listdir(exportPath) if f.split(".")[0][:12] == "sleep_events"]

print(len(files))

1.1.2
2


In [3]:
def whoopSleepStageToStandard(stageName):
    #if stageName == "LATENCY":
    #    return 0
    if stageName == "WAKE":
        return 0
    # if stageName == "DISTURBANCES":
    #    return 0
    if stageName == "LIGHT":
        return 1
    if stageName == "SWS":
        return 2
    if stageName == "REM":
        return 3
    print(stageName)
    return -1

In [4]:
files[0]

'sleep_events (1).csv'

In [6]:
import csv
with open(exportPath + files[0]) as f:
    reader = csv.reader(f)
    lines = [x for x in reader]


In [7]:
row = lines[1:]
row[0][0]

"['2024-03-02T00:31:36.620Z','2024-03-02T00:37:21.728Z')"

In [8]:
rows = lines[1:]
startTime = rows[0][0].split("\'")[1]
startTime

'2024-03-02T00:31:36.620Z'

In [9]:
rows = lines[1:]
endTime = rows[0][0].split("\'")[3]
endTime

'2024-03-02T00:37:21.728Z'

In [10]:
value = whoopSleepStageToStandard(rows[0][1])
value

2

In [19]:
data = []

for sf in files:
    with open(exportPath + sf) as f:
        reader = csv.reader(f)
        lines = [x for x in reader]
        for row in lines[1:]:
            if row[1] in ["LATENCY", "DISTURBANCES"]:
                continue
            toadd = []
            toadd.append(pd.to_datetime(row[0].split("\'")[1]).round('S'))
            toadd.append(pd.to_datetime(row[0].split("\'")[3]).round('S') + datetime.timedelta(seconds=1))
            toadd.append(whoopSleepStageToStandard(row[1]))
            data.append(toadd)

print(len(data))

NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
NO_DATA
24616


In [20]:
data = sorted(data, key=lambda x:x[0])
data[:3]

[[Timestamp('2022-04-11 08:04:24+0000', tz='UTC'),
  Timestamp('2022-04-11 08:09:14+0000', tz='UTC'),
  1],
 [Timestamp('2022-04-11 08:09:14+0000', tz='UTC'),
  Timestamp('2022-04-11 08:18:00+0000', tz='UTC'),
  0],
 [Timestamp('2022-04-11 08:18:00+0000', tz='UTC'),
  Timestamp('2022-04-11 08:19:57+0000', tz='UTC'),
  1]]

In [21]:
# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(data)-1):
    filledRecords.append(data[ri])
    # if there is a gap between this rows end date and the next rows start date
    if data[ri][1] < data[ri+1][0]:
        filledRecords.append([
            data[ri][1],
            data[ri+1][0],
            -1
        ])
filledRecords.append(data[-1])
print(len(filledRecords))

25678


In [22]:
ColumnNames = ["startDate", "endDate", "value"]


whoopHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

whoopHypnoDf = whoopHypnoDf.set_index("startDate")
whoopHypnoDf

,endDate,value
startDate,,
2022-04-11 08:04:24+00:00,2022-04-11 08:09:14+00:00,1
2022-04-11 08:09:14+00:00,2022-04-11 08:18:00+00:00,0
2022-04-11 08:18:00+00:00,2022-04-11 08:19:57+00:00,1
2022-04-11 08:19:57+00:00,2022-04-11 08:27:40+00:00,2
2022-04-11 08:27:40+00:00,2022-04-11 08:33:57+00:00,1
...,...,...
2024-05-10 13:17:31+00:00,2024-05-10 13:49:43+00:00,1
2024-05-10 13:49:43+00:00,2024-05-10 14:10:52+00:00,2
2024-05-10 14:10:52+00:00,2024-05-10 14:34:25+00:00,1


In [23]:
whoopHypnoDf.dtypes

endDate    datetime64[ns, UTC]
value                    int64
dtype: object

In [24]:
writeHypnoDfParquet('whoop', whoopHypnoDf)